# Query Routing

Sometimes its not possible to have all information in a single data source. Maybe your pipeline should be able to handle many different queries, or maybe you have a lot of data and you want to split it into different sources to make it easier find the retrieve the right documents.

Imagine that you have to be able to answer questions about the current weather, but you also want to be able to answer questions about geography. Or you want to be able to limit the search space to search for different categories of documents.

In these cases you can use **Query routing** to dynamically select the right data source to answer the query or search use multiple data sources to answer the query. On way to achieve this is to use the decision making capabilities of LLMs to decide on the fly where to retrieve data from. Query routing is part of the **Pre-retrieval** phase of Advanced RAG.

## Setup libraries and environment

In [ ]:
import os

from dotenv import load_dotenv
from util.helpers import get_wiki_pages, create_and_save_wiki_md_files
from util.query_engines import WeatherQueryEngine

from IPython.display import display, Markdown

from llama_index.core.query_engine import RouterQueryEngine, CustomQueryEngine
from llama_index.core.selectors import LLMMultiSelector, LLMSingleSelector
from llama_index.core.tools import FunctionTool , QueryEngineTool
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, PromptTemplate
from llama_index.llms.openai import OpenAI

In [ ]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio
nest_asyncio.apply()

Add the following to a `.env` file in the root of the project if not already there.

```
OPENAI_API_KEY=<YOUR_KEY_HERE>
```

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Example with `RouterQueryEngine`

In the following example we build a `RouterQueryEngine` that will decide on the fly which data source to use to answer the query. We will have two data sources: 

- **Weather data source** - Query engine that can answer questions using todays weather forecast
- **Wiki pages for cities data source** - Query engine that can answer questions using wikipedia pages for cities

The `RouterQueryEngine` will use a LLM to decide which data source to use to answer the query. We use the `LLMMultiSelector` to allow the LLM to select multiple datasources if it needs to.

#### Define custom `WeatherQueryEngine` 

See implementation of `WeatherQueryEngine` in [query_engines.py](./util/query_engines.py#91)

In [ ]:
weather_query_engine = WeatherQueryEngine(llm=OpenAI(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo"), verbose=True)

In [ ]:
response = await weather_query_engine.aquery("What is the weather in New York?")
print(str(response))

#### Define query engine for Wiki pages for cities

In [ ]:
cities_pages = get_wiki_pages(
    [
        "Aarhus",
        "London",
        "Paris",
        "Berlin",
        "Tokyo",
        "Beijing",
        "Moscow",
        "Sydney",
    ]
)

In [ ]:
create_and_save_wiki_md_files(cities_pages, path="./data/docs/cities/")
cities_documents = SimpleDirectoryReader("./data/docs/cities").load_data()
cities_index = VectorStoreIndex.from_documents(cities_documents, show_progress=True)

#### Define `RouterQueryEngine`

In [ ]:
weather_tool = QueryEngineTool.from_defaults(
    query_engine=weather_query_engine,
    description="Useful for getting todays weather forecast for a given city",
)
cities_tool = QueryEngineTool.from_defaults(
    query_engine=cities_index.as_query_engine(),
    name="Cities Wiki Pages",
    description="Useful for getting information about cities",
)

llm = OpenAI(api_key=OPENAI_API_KEY, model="gpt-4-turbo")
query_engine = RouterQueryEngine(
    selector=LLMMultiSelector.from_defaults(llm=llm),
    llm=llm,
    verbose=True,
    query_engine_tools=[
        weather_tool,
        cities_tool,
    ],
)

The following is the prompt used by the selector:

In [ ]:
DEFAULT_MULTI_SELECT_PROMPT_TMPL = (
    "Some choices are given below. It is provided in a numbered list (1 to {num_choices}), "
    "where each item in the list corresponds to a summary.\n"
    "---------------------\n"
    "{context_list}"
    "\n---------------------\n"
    "Using only the choices above and not prior knowledge, return the top choices "
    "(no more than {max_outputs}, but only select what is needed) that "
    "are most relevant to the question: '{query_str}'\n"
)

In [ ]:
response = await query_engine.aquery("At what time should I go running today in Aarhus?")
display(Markdown(f'{response}'))

In [ ]:
response = await query_engine.aquery("What are the best sights to see in Aarhus?")
display(Markdown(f'{response}'))

In [ ]:
response = await query_engine.aquery(
    "What are the best sights to see in Aarhus and at what time is it best to go there with regards to weather?"
)
display(Markdown(f"{response}"))